# Lab 3: EDA and Data Prep

Exploratory Data Analysis (EDA) is the initial phase of data analysis, focusing on understanding data characteristics, patterns, and relationships through visualizations and summaries. Data Preparation in machine learning involves preprocessing raw data to ensure its suitability for model training, including tasks like handling missing values, encoding variables, scaling features, and splitting data for training and testing. Both EDA and Data Preparation are crucial stages for extracting insights and preparing data for machine learning models.

### Exercise overview

In this exercise, you'll explore the dataset through EDA, uncovering its patterns and relationships. You'll also perform tasks like data preprocessing, handling missing values, and feature engineering to make the data ready for machine learning training.

### Helpful links
- [Data Wrangler in Microsoft Fabric](https://aka.ms/fabric/datawrangler)

## Pre-Requisite

For this Exercise, we expect that you have completed and ran **Lab 1: Access your data through semantic link**. 


## Step 1: Setup your notebook

### Select Lakehouse
First, add the Lakehouse you created from the prior lab exercise.

<br>


![image-alt-text](https://synapseaisolutionsa.blob.core.windows.net/public/Fabric-Conference/add-lakehouse.png)

<br>


### Select environment or install within session

<br>

![Select Environment and then select your environment from the list](https://synapseaisolutionsa.blob.core.windows.net/public/Fabric-Conference/AttachEnv.png)

## Step 2: Load the data as a Spark DataFrame

Load the data that you have created and saved in the lakehouse from the previous notebook.

In [ ]:
df = spark.sql("SELECT * FROM FC_Workshop.churnfromsemanticlink")
display(df)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, ae8554ed-e330-4dd0-b676-6786b9f1ab27)

Convert the spark DataFrame to pandas DataFrame for easier processing and visualization.

In [ ]:
df = df.toPandas()

StatementMeta(, , , Waiting, )

In [ ]:
df.info()

StatementMeta(, , , Waiting, )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerId          10000 non-null  int32  
 1   Surname             10000 non-null  object 
 2   Geography           10000 non-null  object 
 3   Gender              10000 non-null  object 
 4   Age                 10000 non-null  int32  
 5   Tenure              10000 non-null  int32  
 6   Has Credit Card     10000 non-null  object 
 7   Is Active           10000 non-null  object 
 8   Number Of Products  10000 non-null  int32  
 9   Last Credit Score   10000 non-null  int32  
 10  Balance             10000 non-null  float32
 11  EstimatedSalary     10000 non-null  float32
 12  Exited              10000 non-null  bool   
 13  AccountId           10000 non-null  int32  
dtypes: bool(1), float32(2), int32(6), object(5)
memory usage: 713.0+ KB


### Dataset

The dataset contains churn status of 10,000 customers along with 14 attributes that include credit score, geographical location (Germany, France, Spain), gender (male, female), age, tenure (years of being bank's customer), account balance, estimated salary, number of products that a customer has purchased through the bank, credit card status (whether a customer has a credit card or not), and active member status (whether an active bank's customer or not).

The dataset also includes columns such as row number, customer ID, and customer surname that should have no impact on customer's decision to leave the bank. The event that defines the customer's churn is the closing of the customer's bank account, therefore, the column `exit` in the dataset refers to customer's abandonment. Since you don't have much context about these attributes, you'll proceed without having background information about the dataset. Your aim is to understand how these attributes contribute to the `exit` status.

Out of the 10,000 customers, only 2037 customers (around 20%) have left the bank. Therefore, given the class imbalance ratio, it is recommended to generate synthetic data.

- churn.csv

|"CustomerID"|"Surname"|"CreditScore"|"Geography"|"Gender"|"Age"|"Tenure"|"Balance"|"NumOfProducts"|"HasCrCard"|"IsActiveMember"|"EstimatedSalary"|"Exited"|"AccountId"|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|15634602|Hargrave|619|France|Female|42|2|0.00|1|1|1|101348.88|1|4602|
|15647311|Hill|608|Spain|Female|41|1|83807.86|1|0|1|112542.58|0|8401|




## Step 3: Perform Exploratory Data Analysis

Explore the raw data with `display`, do some basic statistics and show chart views. You first need to import required libraries for data visualization such as `seaborn` which is a Python data visualization library to provide a high-level interface for building visuals on dataframes and arrays. Learn more about [`seaborn`](https://seaborn.pydata.org/). 

In [ ]:
import seaborn as sns
sns.set_theme(style="whitegrid", palette="tab10", rc = {'figure.figsize':(9,6)})
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import rc, rcParams
import numpy as np
import pandas as pd
import itertools

StatementMeta(, , , Waiting, )

### Display raw data

In [ ]:
display(df, summary=True)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, ae118d5c-93b3-4975-9bad-1aa3b78b76eb)

### Use Data Wrangler to perform initial data cleansing

To explore and transform any pandas Dataframes in your notebook, launch Data Wrangler directly from the notebook.

1. Under the notebook ribbon Data tab, select Launch Data Wrangler. You'll see a list of activated pandas DataFrames available for editing.
2. Select the DataFrame you wish to open in Data Wrangler. Since this notebook only contains one DataFrame, `df`, select `df`.

<br>

<img src="https://sdkstorerta.blob.core.windows.net/churnblob/select_datawrangler.png"  width="40%" height="10%" title="Screenshot shows where to access the Data Wrangler.">

<br>

Data Wrangler launches and generates a descriptive overview of your data. The table in the middle shows each data column. The Summary panel next to the table shows information about the DataFrame. When you select a column in the table, the summary updates with information about the selected column. In some instances, the data displayed and summarized will be a truncated view of your DataFrame. When this happens, you'll see warning image in the summary pane. Hover over this warning to view text explaining the situation.

Each operation you do can be applied in a matter of clicks, updating the data display in real time and generating code that you can save back to your notebook as a reusable function.

The rest of this section walks you through the steps to perform data cleaning with Data Wrangler.

##### Drop duplicate rows

On the left panel is a list of operations (such as `Find and replace`, `Format`, `Formulas`, `Numeric`) you can perform on the dataset.

1. Expand `Find and replace` and select `Drop duplicate rows`.

2. A panel appears for you to select the list of columns you want to compare to define a duplicate row. Select `CustomerId`, and `AccountId`.

In the middle panel is a preview of the results of this operation. Under the preview is the code to perform the operation. In this instance, the data appears to be unchanged. But since you're looking at a truncated view, it's a good idea to still apply the operation.

3. Select `Apply` (either at the side or at the bottom) to go to the next step.

##### Drop rows with missing data

Use Data Wrangler to drop rows with missing data across all columns.

1. Select `Drop missing values` from `Find and replace`.

2. Choose `Select all` from the `Target columns`.

3. Select `Apply` to go on to the next step.

##### Drop columns
Use Data Wrangler to drop columns that you don't need.

1. Expand `Schema` and select `Drop columns`.

2. Select `CustomerId`, `Surname`, and `AccountId`. These columns appear in red in the preview, to show they're changed by the code (in this case, dropped.)

3.Select `Apply` to go on to the next step.

##### Add code to notebook

Each time you select `Apply`, a new step is created in the `Cleaning steps` panel on the bottom left. At the bottom of the panel, select `Preview code for all steps` to view a combination of all the separate steps.

Select `Add code to notebook` at the top left to close Data Wrangler and add the code automatically. The `Add code to notebook` wraps the code in a function, then calls the function.


>[!NOTE]
>Data Wrangler can not be opened while the notebook kernel is busy. The cell execution must complete prior to launching Data Wrangler. [Learn more about Data Wrangler](https://aka.ms/fabric/datawrangler).

>[!NOTE]
>The code generated by Data Wrangler won't be applied until you manually run the new cell.

<br>

<img style="float: left;" src="https://sdkstorerta.blob.core.windows.net/churnblob/menu_datawrangler.png"  width="45%" height="10%" title="Screenshot shows Data Wrangler menu."> 
<img style="float: left;" src="https://sdkstorerta.blob.core.windows.net/churnblob/missing_data_datawrangler.png"  width="45%" height="10%" title="Screenshot shows Data Wrangler missing data display.">

##### <span style="color:red;"> Exercise 1 </span> 

In [ ]:
# TODO: Add the code generated by Data Wrangler from the above data cleaning steps

### 

### Unlocking Insights with EDA and Visualization

##### Determine attributes

Use this code to determine categorical, numerical, and target attributes.

In [ ]:
# Determine the dependent (target) attribute
dependent_variable_name = "Exited"
print(dependent_variable_name)
# Determine the categorical attributes
categorical_variables = [col for col in df_clean.columns if col in "O"
                        or df_clean[col].nunique() <=5
                        and col not in "Exited"]
print(categorical_variables)
# Determine the numerical attributes
numeric_variables = [col for col in df_clean.columns if df_clean[col].dtype != "object"
                        and df_clean[col].nunique() >5]
print(numeric_variables)

StatementMeta(, , , Waiting, )

Exited


NameError: name 'df_clean' is not defined

Exited



##### The five-number summary 

Show the five-number summary (the minimum score, first quartile, median, third quartile, the maximum score) for the numerical attributes, using box plots.

In [ ]:
df_num_cols = df_clean[numeric_variables]
sns.set(font_scale = 0.7) 
fig, axes = plt.subplots(nrows = 2, ncols = 3, gridspec_kw =  dict(hspace=0.3), figsize = (17,8))
fig.tight_layout()
for ax,col in zip(axes.flatten(), df_num_cols.columns):
    sns.boxplot(x = df_num_cols[col], color='green', ax = ax)
# fig.suptitle('visualize and compare the distribution and central tendency of numerical attributes', color = 'k', fontsize = 12)
fig.delaxes(axes[1,2])


##### Distribution of exited and non-exited customers 

Show the distribution of exited versus non-exited customers across the categorical attributes.

In [ ]:
attr_list = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember', 'NumOfProducts', 'Tenure']
fig, axarr = plt.subplots(2, 3, figsize=(15, 4))
for ind, item in enumerate (attr_list):
    sns.countplot(x = item, hue = 'Exited', data = df_clean, ax = axarr[ind%2][ind//2])
fig.subplots_adjust(hspace=0.7)

##### Distribution of numerical attributes

Show the the frequency distribution of numerical attributes using histogram.


In [ ]:
columns = df_num_cols.columns[: len(df_num_cols.columns)]
fig = plt.figure()
fig.set_size_inches(18, 8)
length = len(columns)
for i,j in itertools.zip_longest(columns, range(length)):
    plt.subplot((length // 2), 3, j+1)
    plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
    df_num_cols[i].hist(bins = 20, edgecolor = 'black')
    plt.title(i)
# fig = fig.suptitle('distribution of numerical attributes', color = 'r' ,fontsize = 14)
plt.show()

#### Summary of observations from the exploratory data analysis

- Most of the customers are from France comparing to Spain and Germany, while Spain has the lower churn rate comparing to France and Germany.
- Most of the customers have credit cards.
- There are customers whose age and credit score are above 60 and below 400, respectively, but they can't be considered as outliers.
- Very few customers have more than two of the bank's products.
- Customers who aren't active have a higher churn rate.
- Gender and tenure years don't seem to have an impact on customer's decision to close the bank account.

### Perform feature engineering 

The following feature engineering generates new attributes based on current attributes.

In [ ]:
df_clean["NewTenure"] = df_clean["Tenure"]/df_clean["Age"]
df_clean["NewCreditsScore"] = pd.qcut(df_clean['CreditScore'], 6, labels = [1, 2, 3, 4, 5, 6])
df_clean["NewAgeScore"] = pd.qcut(df_clean['Age'], 8, labels = [1, 2, 3, 4, 5, 6, 7, 8])
df_clean["NewBalanceScore"] = pd.qcut(df_clean['Balance'].rank(method="first"), 5, labels = [1, 2, 3, 4, 5])
df_clean["NewEstSalaryScore"] = pd.qcut(df_clean['EstimatedSalary'], 10, labels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

### Use Data Wrangler to perform one-hot encoding

Data Wrangler can also be used to perform one-hot encoding. To do so, re-open Data Wrangler. This time, select the `df_clean` data.

1. Expand `Formulas` and select `One-hot encode`.
2. A panel appears for you to select the list of columns you want to perform one-hot encoding on. Select `Geography` and `Gender`.

You could copy the generated code, close Data Wrangler to return to the notebook, then paste into a new cell. Or, select `Add code to notebook` at the top left to close Data Wrangler and add the code automatically.

<br>

<img style="float: left;" src="https://sdkstorerta.blob.core.windows.net/churnblob/1hotencoding_data_wrangler.png"  width="45%" height="20%" title="Screenshot shows one-hot encoding in the Data Wrangler"> 
<img style="float: left;" src="https://sdkstorerta.blob.core.windows.net/churnblob/1hotencoding_selectcolumns_data_wrangler.png"  width="45%" height="20%" title="Screenshot shows selection of columns in the Data Wrangler.">





##### <span style="color:red;"> Exercise 2 </span> 

In [ ]:
# TODO: Add the code generated by Data Wrangler from the above one-hot encoding step

Save the cleaned data to the lakehouse to be used for next step. 

In [ ]:
table_name = "churn_data_clean"
# Create PySpark DataFrame from Pandas
sparkDF=spark.createDataFrame(df_clean1) 
sparkDF.write.mode("overwrite").format("delta").save(f"Tables/{table_name}")
print(f"Spark dataframe saved to delta table: {table_name}")